In [ ]:
from lxml import etree
from cssselect import GenericTranslator
from lxml.etree import XPath
import lxml.html
from urllib.parse import urljoin, urlparse
import requests
import logging
import re
from IPython.display import Markdown
import importlib
import tempfile
from IPython.display import display, HTML

In [ ]:
import sys, os
head,tail=os.path.split(os.getcwd())
sys.path.append(head)

In [ ]:
with open("../logs/parse_fail_20210907-1534.txt") as f:
    lines = f.readlines()
    print("".join(lines[:2]))
    document = "".join(lines[2:])
    #document = "".join(lines)
    
    
with open("../tmp.html","w+") as tf:
    tf.write(document)
    print(f'{tf.name}')
    #display(HTML(f'<a href="{tf.name}">HTML version</a>'))

In [ ]:
import crawler.reviews_parser as rp
importlib.reload(rp)
rp.logger.setLevel(logging.DEBUG)
pass

In [ ]:
rp.extract_recommended(document)

# Don't run below this line

In [ ]:

#Recommended reviews
#CSS selectors
sel_review_container = XPath(GenericTranslator().css_to_xpath("section"))
sel_review_list = XPath(GenericTranslator().css_to_xpath('ul.lemon--ul__373c0__1_cxs.undefined.list__373c0__2G8oH'))
sel_user_card = XPath(GenericTranslator().css_to_xpath('div.lemon--div__373c0__1mboc'))
sel_rating = XPath('descendant-or-self::div/@aria-label')
sel_elite = XPath('descendant-or-self::a[@href="/elite"]')
sel_review = XPath(GenericTranslator().css_to_xpath('span.lemon--span__373c0__3997G.raw__373c0__3rKqk'))
sel_review_count = XPath(GenericTranslator().css_to_xpath('div.nowrap__373c0__35McF > p'))
sel_date = XPath(GenericTranslator().css_to_xpath('div.lemon--div__373c0__1mboc.arrange-unit__373c0__o3tjT.arrange-unit-fill__373c0__3Sfw1.border-color--default__373c0__3-ifU > span.lemon--span__373c0__3997G.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-'))

#Regexes
review_count_re = re.compile(r"(\d+) reviews?")
rating_re = re.compile(r"(\d) star rating")

In [ ]:
business_url = "https://www.yelp.com/biz/dunkin-arlington-heights-2"

In [ ]:
stripped_business_url = urljoin(business_url, urlparse(business_url).path)
response = requests.get(stripped_business_url, params={"start": "120"})
document = response.content

In [ ]:
with open("../logs/parse_fail_20200806-1025.txt") as f:
    lines = f.readlines()
    print("".join(lines[:2]))
    document = "".join(lines[2:])
    #document = "".join(lines)

In [ ]:
document

In [ ]:
root = etree.HTML(document)


if "Hey there trendsetter! You could be the first review for" in root.xpath("string()"):
    print("No reviews")
    review_count = 0
else:
    review_count = sel_review_count(root)[0].xpath("string()")
    review_count = int(review_count_re.match(review_count).group(1))
print(review_count)

print("Elites:")
print(XPath('//a[@href="https://www.yelp.com/elite"]')(root))
print("---------")

#Grab all reviews
review_list = None
for elem in sel_review_container(root):
    #print(elem.xpath("string()"))
    if "Recommended Reviews" in elem.xpath("string()"):
        review_list = sel_review_list(elem)[0]
        #print("Found")
    #print("\n\n")

assert review_list is not None

#print(review_list.xpath("string()"))
        
review_accum = []

#Iterate reviews
for review in review_list:
    assert review.tag == "li"

    #Drop down one level
    review = review[0]
    assert review.tag == "div"

    #Grab user card
    user_card = review[0][0][0][0]

    #Account info
    data_container = user_card[1]
    name_and_loc = data_container[0]
    
    print(name_and_loc.xpath("string()"))
    if "Qype User" in name_and_loc.xpath("string()"):
        print("Qype User, skipping")
        continue
    
    name = name_and_loc[0].xpath("string()")
    location = name_and_loc[1].xpath("string()")
    stats = data_container[1]
    stats_list = list(stats.iter("b"))

    friends = stats_list[0].text
    reviews = stats_list[1].text
    if len(stats_list) > 2:
        photos = stats_list[2].text
    else:
        photos = "0"
        
    print(sel_elite(user_card))
    if len(sel_elite(user_card)) != 0:
        elite = True
        print("Found elite: %s" % name)
    else:
        elite = False

    #Profile picture
    picture_container = user_card[0]
    user_image_url = list(picture_container.iter("img"))[0].get("src")
    user_page_url = list(picture_container.iter("a"))[0].get("href")

    #Grab the rating and review
    review_card = review[1]
    rating = str(sel_rating(review_card)[0])
    rating = int(rating_re.match(rating).group(1))
    review = sel_review(review_card)[0].xpath("text()")
    review = '\n'.join(review)

    #Save the data
    review_accum.append({
        "content": review,
        "rating": rating,
        "user_image_url": user_image_url,
        "user_page_url": user_page_url,
        "user_name": name,
        "user_location": location,
        "user_friends": friends,
        "user_review_count": reviews,
        "user_photos": photos,
        })

In [ ]:
review_accum

# Filtered

In [ ]:
business_url = "https://www.yelp.com/not_recommended_reviews/mustache-bills-diner-barnegat-light"
stripped_business_url = urljoin(business_url, urlparse(business_url).path)
response = requests.get(stripped_business_url)
document = response.content

In [ ]:
with open("../logs/parse_fail_20200719-0639.txt") as f:
    lines = f.readlines()
    print("".join(lines[:2]))
    document = "".join(lines[2:])

In [ ]:
#CSS selectors -- filtered
sel_filtered_review_container = XPath(GenericTranslator().css_to_xpath("div.ysection.not-recommended-reviews.review-list-wide div.review.review--with-sidebar"))
sel_filtered_review_count = XPath(GenericTranslator().css_to_xpath("div.ysection.not-recommended-reviews.review-list-wide > h3"))
sel_filtered_user_card = XPath(GenericTranslator().css_to_xpath("div.ypassport.media-block"))
sel_filtered_user_img = XPath(GenericTranslator().css_to_xpath("img.photo-box-img"))
sel_filtered_user_name = XPath(GenericTranslator().css_to_xpath("span.user-display-name"))
sel_filtered_user_location = XPath(GenericTranslator().css_to_xpath("li.user-location"))
sel_filtered_user_friends = XPath(GenericTranslator().css_to_xpath("li.friend-count > b"))
sel_filtered_user_reviews = XPath(GenericTranslator().css_to_xpath("li.review-count > b"))
sel_filtered_user_photos = XPath(GenericTranslator().css_to_xpath("li.photo-count > b"))
sel_filtered_rating = XPath(GenericTranslator().css_to_xpath("img[alt $= 'rating']"))
sel_filtered_date = XPath(GenericTranslator().css_to_xpath("span.rating-qualifier"))
sel_filtered_review_card = XPath(GenericTranslator().css_to_xpath("div.review-content"))
sel_filtered_content = XPath(GenericTranslator().css_to_xpath("p"))
sel_filtered_content_toggleable = XPath(GenericTranslator().css_to_xpath("span.js-content-toggleable.hidden"))



#Regexes
filtered_review_count_re = review_count_re
filtered_rating_re = re.compile(r"(\d).0 star rating")

In [ ]:
def extract_filtered(document):
    root = etree.HTML(document)

    review_count = sel_filtered_review_count(root)[0].xpath("string()")
    review_count = int(filtered_review_count_re.search(review_count).group(1))

    review_accum = []

    #Iterate reviews
    for review in sel_filtered_review_container(root):
        display(Markdown("---"))

        #Grab user card
        user_card = sel_filtered_user_card(review)[0]

        if "Qype User" in user_card.xpath("string()"):
            print("Skipping Qype user")
            continue

        #Account info
        name_container = sel_filtered_user_name(user_card)[0]

        #Profile picture
        picture_container = sel_filtered_user_img(user_card)[0]
        user_image_url = picture_container.get("src")
        
        name = name_container.xpath("string()").strip()
        hovercard_id = name_container.get("data-hovercard-id")
        try:
            location = sel_filtered_user_location(user_card)[0].xpath("string()").strip()
        except:
            locaiton = None
        
        friends = sel_filtered_user_friends(user_card)[0].text
        reviews = sel_filtered_user_reviews(user_card)[0].text
        try:
            photos = sel_filtered_user_photos(user_card)[0].text
        except:
            photos = 0
            
        #print(user_card.xpath("string()"))
        #print(review.xpath("string()"))
        
        review_card = sel_filtered_review_card(review)[0]

        #Grab the rating and review
        rating = sel_filtered_rating(review_card)[0].get("alt")
        rating = int(filtered_rating_re.match(rating).group(1))
        date = sel_filtered_date(review_card)[0].text.strip()
        
        review_container = None
        review_containers = sel_filtered_content_toggleable(review_card)
        
        review_containers_str = '\n\n\n'.join((e.xpath('string()') for e in review_containers))
        
        print(f"Review containers:\n{len(review_containers)}\n{review_containers_str}")
        assert len(review_containers) <= 1
        if len(review_containers) == 1:
            review_container = review_containers[0]
        else:
            review_container = sel_filtered_content(review_card)[0]
        
        review = review_container.xpath("text()")
        review = (line.strip() for line in review)
        review = '\n'.join(review)
        
        #print(name,friends,reviews,rating,date)

        #Save the data
        review_accum.append({
            "content": review,
            "rating": rating,
            "date": date,
            "user_image_url": user_image_url,
            "data_hovercard_id": hovercard_id,
            "user_name": name,
            "user_location": location,
            "user_friends": friends,
            "user_review_count": reviews,
            "user_photos": photos,
            })
    return review_accum, review_count

In [ ]:
extract_filtered(document)

In [ ]:
root = etree.HTML(document)
review_count = sel_filtered_review_count(root)[0].xpath("string()")

In [ ]:
review_count